This file is part of GaPSE
Copyright (C) 2022 Matteo Foglieni

GaPSE is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

GaPSE is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
General Public License for more details.

You should have received a copy of the GNU General Public License
along with GaPSE. If not, see <http://www.gnu.org/licenses/>.
```


```

# Practical example

This notebook provides a practival example of how compute all the Two-Point Correlation Functions (TPCFs) concerninf the Luminosity Distance (LD) preturbations and relativistic Galaxy Number Counts (GNC). They are splitted in:
- 16 TPCFs for LDxLD
- 25 TPCFs for GNCxGNC
- 20 TPCFs for GNCxLD
- 20 TPCFs for LDxGNC

## Basics and Cosmology parameters implementation

In [1]:
PATH_TO_GAPSE = "../";

In [2]:
include(PATH_TO_GAPSE * "src/GaPSE.jl");

In [3]:
using DelimitedFiles, Plots, Dierckx, SpecialFunctions
using Printf, LaTeXStrings

In [4]:
FILE_BACKGROUND = PATH_TO_GAPSE * "data/WideA_ZA_background.dat"

z_min, z_max, θ_max = 0.05, 0.20, π / 2.0
params = GaPSE.CosmoParams(z_min, z_max, θ_max;
    Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70, s_lim=1e-2,
    s_b = 0.0, 𝑓_evo = 0.0, b = 1.0,
    IPS_opts = Dict(
        :fit_left_min => 1e-6, :fit_left_max => 3e-6,
        :fit_right_min => 1e1, :fit_right_max => 2e1),
    IPSTools_opts = Dict(
        :N => 1024, :fit_min => 0.05, :fit_max => 0.5, 
        :con => true, :k_min => 1e-8, :k_max => 10.0),
    WFI_opts = Dict(
        :llim => 0.0, :rlim => Inf, :N => 1000,
         :trap => true, :rtol => 1e-2, :atol => 0.0,
         :ss_start => 0.0, :ss_step => 10.0,
         :ss_stop => 0.0),
);

## Window F and Integrated window F computation

In [5]:
calc_xs = [x for x in 0:0.01:5];
calc_μs = union([μ for μ in -1.0:0.01:(-0.9)], 
        [μ for μ in -0.9:0.05:0.9], 
        [μ for μ in 0.9:0.01:1.0]);

name_F = "data/F.txt"

GaPSE.print_map_F(PATH_TO_GAPSE * name_F, 
    calc_xs, calc_μs; trapz = true, 
    Fmap_opts = Dict(
        :θ_max => θ_max, :tolerance => 1e-8, 
        :N => 1000, :pr => true,
        #:rtol => 1e-2, :atol => 1e-3,
    )
)

window F evaluation: 100%|██████████████████████████████| Time: 0:24:23


In [6]:
new_calc_μs = union([μ for μ in -1.0:0.01:(-0.9)], 
        [μ for μ in -0.9:0.05:0.9], 
        [μ for μ in 0.9:0.01:1.0]);
name_intF = "data/IntF.txt"

GaPSE.print_map_IntegratedF(
    PATH_TO_GAPSE * name_F, 
    PATH_TO_GAPSE * name_intF, 
    z_min, z_max, new_calc_μs, FILE_BACKGROUND;
    trap = true, Dict(
        :ss_start => 100, :ss_stop => 500, 
        :ss_step => 50, :llim => 0.0, :rlim => Inf, 
        :rtol => 5e-2, :atol => 0.0, :N => 300, :pr => true,
    )...
)

calculating intF: 100%|█████████████████████████████████| Time: 0:00:09


## Cosmology implementation

In [7]:
#FILE_F_MAP = PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt"
#FILE_IF_MAP = PATH_TO_GAPSE * "data/IntegrF_REFERENCE_pi2.txt"
FILE_PS = PATH_TO_GAPSE * "test/datatest/file_pk.txt"

cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, 
    PATH_TO_GAPSE * name_F, PATH_TO_GAPSE * name_intF)

GaPSE.parameters_used(stdout, cosmo)

###############
#    GaPSE    #
############### 
#
# The Cosmology considered had the following paremeters:
# 
# - Matter Power Spectrum input file: "../test/datatest/file_pk.txt"
# - F window function input file: "../data/F.txt"
# - Background data input file: "../data/WideA_ZA_background.dat"
#
# - Basic CosmoParams considered: 
#	 z_min = 0.05 	 z_max = 0.2
#	 θ_max = 1.5707963267948966 [rad] 	 h_0 = 0.7
#	 Ω_b = 0.0489 	 Ω_cdm = 0.25102 	 Ω_M0 = 0.29992
#	 b = 1.0 	 𝑓_evo = 0.0 	 s_b = 0.0
#
# - CosmoParams about the Input Power Spectrum: 
#	 fit_left_min = 1.0e-6 	 fit_right_min = 10.0 	 
#	 fit_left_max = 3.0e-6 	 fit_right_max = 20.0 	 
#
# - CosmoParams about the Input Power Spectrum Tools: 
#	 k_max = 10.0 	 fit_max = 0.5 	 N = 1024 	 
#	 con = true 	 k_min = 1.0e-8 	 fit_min = 0.05 	 
#
# - CosmoParams about the Integrated Window Function F: 
#	 ss_step = 10.0 	 N = 1000 	 rtol = 0.01 	 
#	 llim = 0.0 	 atol = 0.0 	 rlim = Inf 	 
#	 ss_stop = 0.0 	 trap = true 	 ss_start = 0.

## LD TPCFs Computation 

In [10]:
GaPSE.print_map_sum_ξ_LD_multipole(
    cosmo, "map_sum_xis_LD_L0_noF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = false, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

auto_doppler, L=0: 100%|████████████████████████████████| Time: 0:00:00
auto_lensing, L=0: 100%|████████████████████████████████| Time: 0:01:59
auto_localgp, L=0: 100%|████████████████████████████████| Time: 0:00:00
auto_integratedgp, L=0: 100%|███████████████████████████| Time: 0:00:09
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
doppler_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:01
integratedgp_doppler, L=0: 100%|████████████████████████| Time: 0:00:01
lensing_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:02
localgp_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:01
lensing_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:12
integratedgp_lensing, L=0: 100%|████████████████████████| Time: 


time needed for map_sum_ξ_LD_multipole [in s] = 173.60419 



In [11]:
GaPSE.print_map_sum_ξ_LD_multipole(
    cosmo, "map_sum_xis_LD_L0_withF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = true, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

auto_lensing, L=0: 100%|████████████████████████████████| Time: 0:01:36
auto_integratedgp, L=0: 100%|███████████████████████████| Time: 0:00:13
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
doppler_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:01
integratedgp_doppler, L=0: 100%|████████████████████████| Time: 0:00:01
lensing_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:02
localgp_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:01
lensing_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:24
integratedgp_lensing, L=0: 100%|████████████████████████| Time: 0:00:23
localgp_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:01
integratedgp_localgp, L=0: 100%|████████████████████████| Time: 


time needed for map_sum_ξ_LD_multipole [in s] = 174.32165 



## GNC TPCFs Computation 

In [12]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, "map_sum_xis_GNC_L0_noF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = false, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

auto_newton, L=0: 100%|█████████████████████████████████| Time: 0:00:00
auto_doppler, L=0: 100%|████████████████████████████████| Time: 0:00:00
auto_lensing, L=0: 100%|████████████████████████████████| Time: 0:02:33
auto_localgp, L=0: 100%|████████████████████████████████| Time: 0:00:00
auto_integratedgp, L=0: 100%|███████████████████████████| Time: 0:00:09
newton_doppler, L=0: 100%|██████████████████████████████| Time: 0:00:00
doppler_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_lensing, L=0: 100%|██████████████████████████████| Time: 0:00:08
lensing_newton, L=0: 100%|██████████████████████████████| Time: 0:00:06
newton_localgp, L=0: 100%|██████████████████████████████| Time: 0:00:00
localgp_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_integratedgp, L=0: 100%|█████████████████████████| Time: 0:00:01
integratedgp_newton, L=0: 100%|█████████████████████████| Time: 0:00:01
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 


time needed for map_sum_ξ_GNC_multipole [in s] = 231.69391 



In [13]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, "map_sum_xis_GNC_L0_withF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = true, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

auto_newton, L=0: 100%|█████████████████████████████████| Time: 0:00:00
auto_lensing, L=0: 100%|████████████████████████████████| Time: 0:01:39
auto_integratedgp, L=0: 100%|███████████████████████████| Time: 0:00:13
newton_doppler, L=0: 100%|██████████████████████████████| Time: 0:00:00
doppler_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_lensing, L=0: 100%|██████████████████████████████| Time: 0:00:05
lensing_newton, L=0: 100%|██████████████████████████████| Time: 0:00:05
newton_integratedgp, L=0: 100%|█████████████████████████| Time: 0:00:01
integratedgp_newton, L=0: 100%|█████████████████████████| Time: 0:00:02
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
doppler_integratedgp, L=0: 100%|████████████████████████| Time: 


time needed for map_sum_ξ_GNC_multipole [in s] = 200.53506 



## GNCxLD TPCFs Computation 

In [14]:
GaPSE.print_map_sum_ξ_GNCxLD_multipole(
    cosmo, "map_sum_xis_GNCxLD_L0_noF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = false, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

newton_doppler, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_lensing, L=0: 100%|██████████████████████████████| Time: 0:00:08
newton_localgp, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_integratedgp, L=0: 100%|█████████████████████████| Time: 0:00:01
doppler_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
doppler_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:01
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
lensing_lensing, L=0: 100%|█████████████████████████████| Time: 0:01:59
lensing_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:02
lensing_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:12
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
localgp_lensing, L=0: 100%|█████████████████████████████| Time: 


time needed for map_sum_ξ_GNCxLD_multipole [in s] = 187.83012 



In [15]:
GaPSE.print_map_sum_ξ_GNCxLD_multipole(
    cosmo, "map_sum_xis_GNCxLD_L0_withF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = true, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

newton_doppler, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_lensing, L=0: 100%|██████████████████████████████| Time: 0:00:05
newton_integratedgp, L=0: 100%|█████████████████████████| Time: 0:00:01
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
doppler_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:01
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
lensing_lensing, L=0: 100%|█████████████████████████████| Time: 0:01:52
lensing_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:02
lensing_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:24
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
localgp_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
localgp_integratedgp, L=0: 100%|████████████████████████| Time: 0:00:01
integratedgp_doppler, L=0: 100%|████████████████████████| Time: 


time needed for map_sum_ξ_GNCxLD_multipole [in s] = 200.73078 



## LDxGNC TPCFs Computation 

In [16]:
GaPSE.print_map_sum_ξ_LDxGNC_multipole(
    cosmo, "map_sum_xis_LDxGNC_L0_noF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = false, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

doppler_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
lensing_newton, L=0: 100%|██████████████████████████████| Time: 0:00:06
localgp_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
integratedgp_newton, L=0: 100%|█████████████████████████| Time: 0:00:01
doppler_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
integratedgp_doppler, L=0: 100%|████████████████████████| Time: 0:00:01
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
lensing_lensing, L=0: 100%|█████████████████████████████| Time: 0:01:59
localgp_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:01
integratedgp_lensing, L=0: 100%|████████████████████████| Time: 0:00:13
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
lensing_localgp, L=0: 100%|█████████████████████████████| Time: 


time needed for map_sum_ξ_LDxGNC_multipole [in s] = 182.48000 



In [17]:
GaPSE.print_map_sum_ξ_LDxGNC_multipole(
    cosmo, "map_sum_xis_LDxGNC_L0_withF.txt", 
    10 .^ range(0, 3, length = 300);
    L = 0, use_windows = true, 
    enhancer = 1e8, single = true, N_μs = 30, μ_atol = 0.0, 
    μ_rtol = 1e-2, N_χs = 100, N_χs_2 = 60);

doppler_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
lensing_newton, L=0: 100%|██████████████████████████████| Time: 0:00:05
integratedgp_newton, L=0: 100%|█████████████████████████| Time: 0:00:02
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
localgp_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:00
integratedgp_doppler, L=0: 100%|████████████████████████| Time: 0:00:01
doppler_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:02
lensing_lensing, L=0: 100%|█████████████████████████████| Time: 0:01:35
localgp_lensing, L=0: 100%|█████████████████████████████| Time: 0:00:01
integratedgp_lensing, L=0: 100%|████████████████████████| Time: 0:00:23
doppler_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:00
lensing_localgp, L=0: 100%|█████████████████████████████| Time: 0:00:03
integratedgp_localgp, L=0: 100%|████████████████████████| Time: 0:00:01
doppler_integratedgp, L=0: 100%|████████████████████████| Time: 


time needed for map_sum_ξ_LDxGNC_multipole [in s] = 184.28141 

